**Task Details:**
Cleaning of City Name in Guftugu Publications Dataset

# Loading data

In [ ]:
#Loading of Dataset and required Libraries

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns


dt= pd.read_csv(
    "/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv",
    encoding="utf_8")

import warnings  
warnings.filterwarnings('ignore')

# Basic Data Exploration

In [ ]:
#to check few rows
dt.head()

In [ ]:
#to check the number of column & rows
print("dimensions are : ", dt.shape)

so data contains 19239 rows and 8 columns

In [ ]:
#to check the columns names, data type and null values (if any)

print(dt.info())

"Non-Null Count" of few columns shows the presence of null values

# Data cleaning

**Handling missing values.**

In [ ]:
print(dt.isna().sum())

Three columns contains the missing values. we will drop these.

In [ ]:
print("Before drop, total rows are: ", dt.shape[0])

#drop the null values
dt.dropna(inplace=True)

print("After drop, total rows are: ", dt.shape[0])

print(dt.isna().sum())

# City Names Cleaning

**Adding another dataset with full list of City Names**

In [ ]:
dtc= pd.read_csv("../input/pakistan-city-names/pk-city-v2.csv",encoding='windows-1252')
dtc = dtc.rename(columns = {'City_Name': 'City',}, inplace = False)
dtc

In [ ]:
duplicates = dt.duplicated(subset = "City", keep = False)
len(dt[duplicates])

so out of 19226 rows, 16234 rows have duplicate city names

In [ ]:

dt.City=dt.City.str.split(':',expand=True)
dt.City=dt.City.str.split(',',expand=True)
dt.City=dt.City.str.split('/',expand=True)

print("Total unique City Names in Guftugu Dataset: ", dt["City"].nunique())
print("Total Unique City Names in external Dataset: ", dtc["City"].nunique())

The "City" column contains many typos. lets fix it.

**Using Manual name replacement**

In [ ]:
dt["City"] = dt["City"].str.strip()
dtc["City"] = dtc["City"].str.strip()
dt['City'] = dt['City'].replace("khi",'karachi')
dt['City'] = dt['City'].replace("fsd",'faisalabad')
dt['City'] = dt['City'].replace(['lahore',"lhr"],'lahore')
dt['City'] = dt['City'].replace(["اسلام اباد",'isb',"ict","A-17","A-18","B-17","B-18","C-15","C-16","C-17","C-18","D-10","D-11","D-12","D-13","D-14","D-15","D-16","D-17","D-18","E-7","E-8","E-9","E-10","E-11","E-12","E-13","E-14","E-15","E-16","E-17","E-18","F-5","F-6","F-7","F-8","F-9","F-10","F-11","F-12","F-13","F-14","F-15","F-16","F-17","F-18","G-5","G-6","G-7","G-8","G-9","G-10","G-11","G-12","G-13","G-14","G-15","G-16","G-17","G-18","H-8","H-9","H-10","H-11","H-12","H-13","H-14","H-15","H-16","H-17","H-18","I-8","I-9","I-10","I-11","I-12","I-13","I-14","I-15","I-16","I18","A17","A18","B17","B18","C15","C16","C17","C18","D10","D11","D12","D13","D14","D15","D16","D17","D18","E7","E8","E9","E10","E11","E12","E13","E14","E15","E16","E17","E18","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","G5","G6","G7","G8","G9","G10","G11","G12","G13","G14","G15","G16","G17","G18","H8","H9","H10","H11","H12","H13","H14","H15","H16","H17","H18","I8","I9","I10","I11","I12","I13","I14","I15","I16","I17","I18",],'islamabad')
dt['City'] = dt['City'].replace('rwp','rawalpindi')
print("After manual tweaking total unique City Names in Guftugu Dataset are: ", dt["City"].nunique())

**Using Fuzzy Logic to replace the city name with its corresponding matching name**

In [ ]:
#using fuzzy logic

print("Total unique City Names in Guftugu Dataset BEFORE fuzz logic: ", dt["City"].nunique())
from fuzzywuzzy import process
for city in dtc['City']:
    matches = process.extract(city, dt["City"].unique() , limit = len(dtc['City']))
    for potential_match in matches:
        if potential_match[1] > 80:
                dt.loc[dt['City'] == potential_match[0],"City"] = city
                
print("Total unique City Names in Guftugu Dataset AFTER fuzz logic: ", dt["City"].nunique())

**Unprocessed Names by Fuzz Logic**

In [ ]:
#lets take the list of other cities which are not part of processed by fuzzy

abc=dt[~dt["City"].isin(dtc["City"])]
print("so we have total", len(abc["City"].unique()) ,"Cities which are not processed by fuzzywuzzy. we have to handle these as well.")

In [ ]:
dt.loc[dt["City"].str.len() <= 2] = "Unknown_City"
print("Total unique City Names in Guftugu Dataset NOW: ", dt["City"].nunique())

# Uploading to csv file

In [ ]:
dt.to_csv('pakcitylist.csv',index=False)

# Thanks for viewing my notebook, you are welcome to give any comments/suggestions to further improve my work.